In [7]:
import pandas as pd
import numpy as np

from m4_feats_polars import *
from m5_sb_models import *

In [8]:
lgb_params_1 = {
    'boosting_type': 'gbdt', 
    'metric': 'rmse',
    'reg_alpha': 0.0031, 
    'reg_lambda': 0.001, 
    'colsample_bytree': 0.8,  
    'subsample_freq': 1,  
    'subsample': 0.75,  
    'learning_rate': 0.017, 
    'num_leaves': 19, 
    'min_child_samples': 46,
    'n_estimators': 400,
    'verbosity': -1
    }

param = {'n_estimators': 1024,
         'learning_rate': 0.005,
         'metric': 'rmse',
         'force_col_wise': True,
         'verbosity': 0,}

import polars as pl
data_path     = 'kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(f'{data_path}/train_logs.csv')
test_logs    = pl.scan_csv(f'{data_path}/test_logs.csv')
train_scores = pl.scan_csv(f'{data_path}/train_scores.csv')

In [9]:
Best_Feature_Set = [
    'train_down_events_counts.pkl', 
    'train_vector_one_gram.pkl', 
    'train_create_pauses.pkl', 
    'train_essay_paragraphs.pkl', 
    'train_cursor_pos_acceleration.pkl', 
    'train_word_count_acceleration.pkl', 
    'train_vector_two_gram.pkl']

# (action_time_by_activity, categorical_nunique) # best 2 0.602997
('get_keys_pressed_per_second', 'p_burst_feats', 'r_burst_feats') # best 3 0.602696

best_feature_set_2 = [
    'train_down_events_counts.pkl',
    'train_vector_one_gram.pkl',
    'train_create_pauses.pkl',
    'train_essay_paragraphs.pkl',
    'train_essay_sentences.pkl',
    'train_cursor_pos_acceleration.pkl',
    'train_word_count_acceleration.pkl',
    'train_categorical_nunique.pkl',
    'train_cursor_pos_rate_of_change.pkl',
    'train_get_keys_pressed_per_second.pkl']   

base_feat_3 = ['train_down_events_counts.pkl', 
               'train_vector_one_gram.pkl', 
               'train_create_pauses.pkl', 
               'train_essay_paragraphs.pkl', 
               'train_cursor_pos_acceleration.pkl', 
               'train_essay_sentences.pkl', 
               'train_action_time_baseline_stats.pkl']

In [10]:
# # best_feature_set_2 - DONE UNTIL THE END
# train_essays          = get_essay_df(train_logs.collect().to_pandas())
# test_essays           = get_essay_df(test_logs.collect().to_pandas())

# # train_down_events_counts
# tr_down_events_counts, ts_down_events_counts = down_events_counts(train_logs, test_logs)
# # train_vector_one_gram
# tr_vect_one, ts_vect_one = countvectorize_one_one(train_essays, test_essays)
# # train_create_pauses
# tr_pauses, ts_pauses = create_pauses(train_logs, test_logs)
# # train_cursor_pos_acceleration
# tr_cursor_pos_acc, ts_cursor_pos_acc = cursor_pos_acceleration(train_logs, test_logs)
# # train_word_count_acceleration
# tr_word_count_acc, ts_word_count_acc = word_count_acceleration(train_logs, test_logs)
# # nunique
# tr_nunique, ts_nunique = categorical_nunique(train_logs, test_logs)
# # cursor_pos_rate_of_change
# tr_cur_pos_roc, ts_cur_pos_roc = cursor_pos_rate_of_change(train_logs, test_logs)
# # get keys pressed per second
# tr_get_keys, ts_get_keys = get_keys_pressed_per_second(train_logs.collect().to_pandas(), test_logs.collect().to_pandas())

# # # r_burst_feats
# # tr_r_burst, ts_r_burst = r_burst_feats(train_logs, test_logs)
# # # p_burst_feats
# # tr_p_burst, ts_p_burst = p_burst_feats(train_logs, test_logs)

# train_feats = tr_down_events_counts.join(tr_vect_one, on='id', how='left')
# train_feats = train_feats.join(tr_pauses, on='id', how='left')
# train_feats = train_feats.join(tr_cursor_pos_acc, on='id', how='left')
# train_feats = train_feats.join(tr_word_count_acc, on='id', how='left')
# train_feats = train_feats.join(tr_nunique, on='id', how='left')
# train_feats = train_feats.join(tr_cur_pos_roc, on='id', how='left')
# train_feats = train_feats.join(tr_get_keys, on='id', how='left')

# #train_feats = train_feats.join(tr_vect_two, on='id', how='left')
# #train_feats = train_feats.join(tr_p_burst, on='id', how='left')
# #train_feats = train_feats.join(tr_r_burst, on='id', how='left')
# #train_feats = train_feats.join(tr_nunique, on='id', how='left')


# test_feats = ts_down_events_counts.join(ts_vect_one, on='id', how='left')
# test_feats = test_feats.join(ts_pauses, on='id', how='left')
# test_feats = test_feats.join(ts_cursor_pos_acc, on='id', how='left')
# test_feats = test_feats.join(ts_word_count_acc, on='id', how='left')
# test_feats = test_feats.join(ts_nunique, on='id', how='left')
# test_feats = test_feats.join(ts_cur_pos_roc, on='id', how='left')
# test_feats = test_feats.join(ts_get_keys, on='id', how='left')

# #test_feats = test_feats.join(ts_vect_two, on='id', how='left')
# #test_feats = test_feats.join(ts_p_burst, on='id', how='left')
# #test_feats = test_feats.join(ts_r_burst, on='id', how='left')
# #test_feats = test_feats.join(ts_nunique, on='id', how='left')


# train_logs = train_logs.collect().to_pandas()
# test_logs = test_logs.collect().to_pandas()
# train_scores = train_scores.collect().to_pandas()
# train_feats = train_feats.collect().to_pandas()
# test_feats = test_feats.collect().to_pandas()

# train_feats           = train_feats.merge(parag_feats(train_essays), on='id', how='left')
# test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
# train_feats           = train_feats.merge(sent_feats(train_essays), on='id', how='left')
# test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')

# train_feats           = train_feats.merge(train_scores, on=['id'], how='left')
# print(f'train feats shape {train_feats.shape}')

In [11]:
# best_feature_set_1 - PARTIAL

train_essays          = get_essay_df(train_logs.collect().to_pandas())
test_essays           = get_essay_df(test_logs.collect().to_pandas())

# train_down_events_counts
tr_down_events_counts, ts_down_events_counts = down_events_counts(train_logs, test_logs)
# train_vector_one_gram
tr_vect_one, ts_vect_one = countvectorize_one_one(train_essays, test_essays)
# train_create_pauses
tr_pauses, ts_pauses = create_pauses(train_logs, test_logs)
# train_cursor_pos_acceleration
tr_cursor_pos_acc, ts_cursor_pos_acc = cursor_pos_acceleration(train_logs, test_logs)
# train_word_count_acceleration
tr_word_count_acc, ts_word_count_acc = word_count_acceleration(train_logs, test_logs)
# train_vector_two_gram
tr_vect_two, ts_vect_two = countvectorize_two_one(train_essays, test_essays)


# count_of_activities(train_logs, test_logs)
tr_act_count, ts_act_count = count_of_activities(train_logs, test_logs)
# # r_burst_feats
tr_r_burst, ts_r_burst = r_burst_feats(train_logs, test_logs)
# # p_burst_feats
tr_p_burst, ts_p_burst = p_burst_feats(train_logs, test_logs)
# nunique
tr_nunique, ts_nunique = categorical_nunique(train_logs, test_logs)
# get_keys_pressed_per_second
tr_get_keys, ts_get_keys = get_keys_pressed_per_second(train_logs.collect().to_pandas(), 
                                                       test_logs.collect().to_pandas())

train_feats = tr_down_events_counts.join(tr_vect_one, on='id', how='left')
train_feats = train_feats.join(tr_pauses, on='id', how='left')
train_feats = train_feats.join(tr_cursor_pos_acc, on='id', how='left')
train_feats = train_feats.join(tr_word_count_acc, on='id', how='left')
train_feats = train_feats.join(tr_vect_two, on='id', how='left')
train_feats = train_feats.join(tr_r_burst, on='id', how='left')
train_feats = train_feats.join(tr_nunique, on='id', how='left')
train_feats = train_feats.join(tr_get_keys, on='id', how='left')


test_feats = ts_down_events_counts.join(ts_vect_one, on='id', how='left')
test_feats = test_feats.join(ts_pauses, on='id', how='left')
test_feats = test_feats.join(ts_cursor_pos_acc, on='id', how='left')
test_feats = test_feats.join(ts_word_count_acc, on='id', how='left')
test_feats = test_feats.join(ts_vect_two, on='id', how='left')
test_feats = test_feats.join(ts_r_burst, on='id', how='left')
test_feats = test_feats.join(ts_nunique, on='id', how='left')
test_feats = test_feats.join(ts_get_keys, on='id', how='left')


train_logs = train_logs.collect().to_pandas()
test_logs = test_logs.collect().to_pandas()
train_scores = train_scores.collect().to_pandas()
train_feats = train_feats.collect().to_pandas()
test_feats = test_feats.collect().to_pandas()

train_feats           = train_feats.merge(parag_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')

train_feats           = train_feats.merge(train_scores, on=['id'], how='left')
print(f'train feats shape {train_feats.shape}')

KeyboardInterrupt: 

In [ ]:
# missing_cols = set(train_feats.columns) - set(test_feats.columns)
# missing_cols.remove('score')

# for col in missing_cols:
#     test_feats[col] = np.nan

test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, lgb_params_1)

/root/miniconda3/envs/lrp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less 

Final RMSE over 50: 0.604867. Std 0.8282
RMSE by fold 0.604820. Std 0.0077


Best_Feature_Set
Final RMSE over 50: 0.603377. Std 0.8293
RMSE by fold 0.603282. Std 0.0112

Final RMSE over 50: 0.609901. Std 0.8218
RMSE by fold 0.609733. Std 0.0145

Best_Feature_Set_2
Final RMSE over 50: 0.605864. Std 0.8239
RMSE by fold 0.605801. Std 0.0091

In [ ]:
test_ids = test_feats.id
y_pred = np.mean(test_preds, axis=0)

sub = pd.DataFrame({'id': test_ids, 'score': y_pred})
sub.to_csv('submission.csv', index=False)

NameError: name 'test_feats' is not defined

In [ ]:
sub

,id,score
0,0000aaaa,1.402126
1,4444cccc,1.353844
2,2222bbbb,1.336026
